In [ ]:
import re

In [ ]:
import numpy as np
from gensim.models import word2vec

In [ ]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras import losses

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model, Sequential

In [ ]:
TRAIN_FILE = 'data/ratings_train.txt'
TEST_FILE = 'data/ratings_test.txt'

In [ ]:
# train_reviews = []
# train_labels = []
# test_reviews = []
# test_labels = []

# # Read data
# with open(TRAIN_FILE, 'r') as f:
#     next(f) # header skip
#     for line in f.readlines():
#         _, review, label = line.strip().split('\t')
#         label = 'pos' if label == '1' else 'neg'
#         train_reviews.append((review.strip()))
#         train_labels.append((label.strip()))
        
# with open(TEST_FILE, 'r') as f:
#     next(f) # header skip
#     for line in f.readlines():
#         _, review, label = line.strip().split('\t')
#         label = 'pos' if label == '1' else 'neg'
#         test_reviews.append((review.strip()))
#         test_labels.append((label.strip()))

In [154]:
train_reviews = []
test_reviews = []

In [155]:
# Read data
with open(TRAIN_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, label = line.strip().split('\t')
#         label = 'pos' if label == '1' else 'neg'
        train_reviews.append((review.strip(), label.strip()))
        
with open(TEST_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, label = line.strip().split('\t')
#         label = 'pos' if label == '1' else 'neg'
        test_reviews.append((review.strip(), label.strip()))

In [156]:
def tokenizer(sentence):
    tokens = re.findall(r"[\w]+|[^\s\w]", sentence)
    return tokens

In [157]:
train_reviews = [(tokenizer(review), int(label)) for review, label in train_reviews]#[:30000]
test_reviews = [(tokenizer(review), int(label)) for review, label in test_reviews]#[:10000]

In [158]:
train_reviews[:5]

[(['아', '더빙', '.', '.', '진짜', '짜증나네요', '목소리'], 0),
 (['흠',
   '.',
   '.',
   '.',
   '포스터보고',
   '초딩영화줄',
   '.',
   '.',
   '.',
   '.',
   '오버연기조차',
   '가볍지',
   '않구나'],
  1),
 (['너무재밓었다그래서보는것을추천한다'], 0),
 (['교도소', '이야기구먼', '.', '.', '솔직히', '재미는', '없다', '.', '.', '평점', '조정'], 0),
 (['사이몬페그의',
   '익살스런',
   '연기가',
   '돋보였던',
   '영화',
   '!',
   '스파이더맨에서',
   '늙어보이기만',
   '했던',
   '커스틴',
   '던스트가',
   '너무나도',
   '이뻐보였다'],
  1)]

In [159]:
###

In [160]:
X_train = [review for review, _ in train_reviews]
y_train = [label for _, label in train_reviews]
X_test = [review for review, _ in test_reviews]
y_test = [label for _, label in test_reviews]

In [161]:
model_file = 'w2v_file/300dim_1minwords_4context' # 300dim_1minwords_4context
w2v = word2vec.Word2Vec.load(model_file).wv

In [162]:
w2v.index2word[:30]

['.',
 '!',
 ',',
 '영화',
 '?',
 '~',
 '너무',
 '정말',
 ';',
 '진짜',
 '^',
 '이',
 '"',
 '그냥',
 '왜',
 '이런',
 '더',
 '-',
 '수',
 '영화를',
 '다',
 '잘',
 '좀',
 '보고',
 '그',
 '영화는',
 'ㅋㅋ',
 '영화가',
 "'",
 '봤는데']

In [163]:
w2v['영화']

array([ -6.43690526e-01,  -1.94158211e-01,  -3.01703840e-01,
         3.50169539e-01,   2.74797976e-02,   1.87092438e-01,
         3.09477538e-01,  -5.07408008e-03,   1.46904692e-01,
        -5.98304152e-01,   2.10476369e-01,   6.53053522e-01,
         5.09366617e-02,   5.12826979e-01,   9.82050523e-02,
        -7.52453148e-01,   2.83928305e-01,  -6.14261329e-01,
         5.01769722e-01,   1.24346435e-01,   1.10103353e-03,
         1.02179118e-01,   2.76756763e-01,   1.05631053e-01,
        -2.06416696e-01,  -2.51796693e-01,  -2.65574366e-01,
        -4.43947464e-01,  -3.93502057e-01,  -6.97040379e-01,
         5.76580405e-01,  -8.12861323e-02,   3.68262142e-01,
        -2.25619644e-01,   1.55338198e-01,   2.41847023e-01,
        -4.00947750e-01,  -4.02923673e-01,  -4.59300935e-01,
        -5.33362389e-01,   2.56641477e-01,   9.34801437e-03,
         5.79155743e-01,  -1.85691938e-01,   3.09705049e-01,
        -2.90136725e-01,  -2.70215154e-01,  -1.64578736e-01,
        -4.27612141e-02,

In [164]:
X_train[:50]

[['아', '더빙', '.', '.', '진짜', '짜증나네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터보고',
  '초딩영화줄',
  '.',
  '.',
  '.',
  '.',
  '오버연기조차',
  '가볍지',
  '않구나'],
 ['너무재밓었다그래서보는것을추천한다'],
 ['교도소', '이야기구먼', '.', '.', '솔직히', '재미는', '없다', '.', '.', '평점', '조정'],
 ['사이몬페그의',
  '익살스런',
  '연기가',
  '돋보였던',
  '영화',
  '!',
  '스파이더맨에서',
  '늙어보이기만',
  '했던',
  '커스틴',
  '던스트가',
  '너무나도',
  '이뻐보였다'],
 ['막',
  '걸음마',
  '뗀',
  '3세부터',
  '초등학교',
  '1학년생인',
  '8살용영화',
  '.',
  'ㅋㅋㅋ',
  '.',
  '.',
  '.',
  '별반개도',
  '아까움',
  '.'],
 ['원작의', '긴장감을', '제대로', '살려내지못했다', '.'],
 ['별',
  '반개도',
  '아깝다',
  '욕나온다',
  '이응경',
  '길용우',
  '연기생활이몇년인지',
  '.',
  '.',
  '정말',
  '발로해도',
  '그것보단',
  '낫겟다',
  '납치',
  '.',
  '감금만반복반복',
  '.',
  '.',
  '이드라마는',
  '가족도없다',
  '연기못하는사람만모엿네'],
 ['액션이', '없는데도', '재미', '있는', '몇안되는', '영화'],
 ['왜케',
  '평점이',
  '낮은건데',
  '?',
  '꽤',
  '볼만한데',
  '.',
  '.',
  '헐리우드식',
  '화려함에만',
  '너무',
  '길들여져',
  '있나',
  '?'],
 ['걍인피니트가짱이다', '.', '진짜짱이다', '♥'],
 ['볼때마다',
  '눈물나서',
  '죽겠다90년대의',
  '향수자극',
  '!'

In [166]:
id2word = w2v.index2word
id2word.insert(0,'_PAD_')
dics = set(id2word)

In [167]:
len(id2word)

367592

In [169]:
%%time
# X_train = [[id2word.index(w) for w in seq if w in dics] for seq in X_train]
# X_test = [[id2word.index(w) for w in seq if w in dics] for seq in X_test]
X_train = [[id2word.index(w) for w in seq] for seq in X_train]
X_test = [[id2word.index(w) for w in seq] for seq in X_test]

CPU times: user 39min 15s, sys: 2 s, total: 39min 17s
Wall time: 39min 17s


In [127]:
X_train[4]

[119, 5714, 4, 2, 891, 28445, 585]

In [128]:
[id2word[w] for w in X_train[4]]

['연기가', '돋보였던', '영화', '!', '했던', '커스틴', '너무나도']

In [129]:
w2v['영화']

array([-0.05074778,  0.21507718, -0.03183799,  0.16720751, -0.0790204 ,
        0.05391108,  0.05643592, -0.03931455, -0.05158216, -0.296924  ,
       -0.09710837, -0.01641047, -0.05160336,  0.15071635, -0.07543746,
       -0.09708823, -0.05677411, -0.13775918,  0.22767606, -0.05024914,
        0.05447428,  0.05523868, -0.21673222, -0.09153788, -0.00924428,
       -0.25608647, -0.05020567, -0.15303056, -0.13781276, -0.22126454,
        0.06379262, -0.14465603, -0.16598178,  0.03733333,  0.17430207,
       -0.0805093 , -0.18798523, -0.10484594, -0.07179404,  0.12586612,
        0.04610015,  0.07826741,  0.13975617, -0.23230149,  0.04895   ,
       -0.07084665, -0.05017959, -0.04414334,  0.03180394,  0.11655354,
        0.02398936,  0.04559856,  0.00049805, -0.00106748, -0.09793754,
        0.0718876 ,  0.01963489, -0.20780922, -0.08301318, -0.0709358 ,
       -0.35432804, -0.07187297, -0.13821052, -0.1616706 ,  0.30040568,
       -0.19811532, -0.11101404,  0.02803983,  0.05086677, -0.06

In [170]:
MAX_SEQUENCE_LENGTH = 50
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH) # MAX_SEQUENCE_LENGTH만큼 뒤 기준으로 잘림.
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH) # MAX_SEQUENCE_LENGTH만큼 뒤 기준으로 잘림.

In [171]:
X_train[:5]

array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,     35,    960,      1,      1,     10,
          7295,   1079],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,    671,      1,      1,
             1,   6697, 127834,      1,      1,      1,      1, 127835,
          7533,  35218],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0

In [172]:
y_train[:5]

[0, 1, 0, 0, 1]

In [173]:
# y_train = to_categorical(np.array(y_train)) # label one-hot encoding
# y_test = to_categorical(np.array(y_test)) 

In [174]:
EMBEDDING_DIM = 300

In [175]:
embedding_matrix = np.random.random((len(dics), EMBEDDING_DIM)) 
embedding_matrix # 랜덤값으로 초기화

array([[ 0.97220488,  0.84780664,  0.15826245, ...,  0.48506373,
         0.41006084,  0.12062725],
       [ 0.4369374 ,  0.19033408,  0.79737696, ...,  0.04621886,
         0.34993104,  0.83851775],
       [ 0.94633187,  0.14364005,  0.79189217, ...,  0.78095702,
         0.00419944,  0.41495091],
       ..., 
       [ 0.19013866,  0.7689447 ,  0.4005317 , ...,  0.12742979,
         0.48876207,  0.99704198],
       [ 0.72822297,  0.32098973,  0.42245992, ...,  0.97432972,
         0.62096099,  0.61440905],
       [ 0.93267411,  0.97434715,  0.47880116, ...,  0.89839253,
         0.25232352,  0.61268189]])

In [176]:
embedding_matrix.shape

(367592, 300)

In [177]:
iter_id2word = enumerate(id2word)
next(iter_id2word)
for i, word in iter_id2word:
    embedding_matrix[i] = w2v[word]

In [178]:
DIC_SIZE = len(dics)

In [179]:
# create the model
model = Sequential()
model.add(Embedding(DIC_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=EMBEDDING_DIM, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.05))
model.add(Conv1D(filters=EMBEDDING_DIM, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=EMBEDDING_DIM, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 50, 300)           110277600 
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 50, 300)           270300    
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 10, 300)           0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 10, 300)           270300    
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 2, 300)            0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 2, 300)            270300    
__________

In [180]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=4096)

/Users/chad/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 110277600 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


Train on 150000 samples, validate on 50000 samples
Epoch 1/10
150000/150000 [==============================] - 306s - loss: 0.4874 - acc: 0.7531 - val_loss: 0.3825 - val_acc: 0.8286

- https://elitedatascience.com/keras-tutorial-deep-learning-in-python